DESeq analysis of DamID data, to see what sites are differentially methylated.

From prelim analysis, we know that there are regions that are differentially bound.  What do they look like?  They can be big regions, or small regions, or can be clustered or dispersed, or they the same across tissues.

remove all W's that are not Batch3
TLam interaction analysis with missing line removed, just to make sense of the W issue (W issue is that protein line interaction not significant in W, whereas signicant in C)

Do correlation between different datasets of BaseMean and Log2Change value, to answer the question: are areas of high binding the same across protein-line combos?

Line effect hypotheses:
1) If you imagine a simple binary model where the protein either binds or doesn't.  The base mean if the protein doesn't bind is low.  The peaks of the non-NA regions generally speaking are non-overlapping.  
2) Alternatively, some degree of binding.  Regions of binding relatively fixed.
(Most likely a mix of these two things.)
(Depending on this, we might want to reduce our binding data 1's and 0's when comparing to gene expression.  This might also impact whether we ultimately try to build an HMM, if on/off an HMM is easy)

Simplify data to just non-NA rows and plot log2 fold change across genome.  When you look at the difference between lines if you wnat to plot log2fold change across genome.  Send to png.   

We tested whether an interaction was significant by doing a NBLRT test.  Pull the regions that are significant from the created dataframes.
Write a function that take a row number as input, makes a plot.
1) I write to file significant Line by Protein regions (only keep pvalue, padj)
2) I can select regions with significant line by protein interaction, and draw a plot of the log2change from my individual analysis (across the dfiferent lines)

Make a Manhattan plot of p-values across genomes (on raw pvalues).  color those with padj<.05 in diff color.  Do these significant interactions tend to be clustered or dispersed?

Do ACF for these log2fold change for one line by protein tissue combination, or for the main effect of protein across pooled data.




In [1]:
library(DESeq2)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges


Load the files, see what they look like

In [2]:
#This file is the AdRp file, with columns with counts less than 500000 removed, and with rows with rDNA or MtDNA removed
damid<-read.csv("AdRp_noMtDNA")
loc<-read.csv("Locations")
metadata<-read.csv("file_names")

In [3]:
head(loc)
length(loc)

,X,X0,X1,X2
1,0,211000022278049,390,394
2,1,211000022278049,537,541
3,2,211000022278049,573,577
4,3,211000022278049,837,841
5,4,211000022278049,1613,1617
6,5,211000022278049,1618,1622


[1] 4

have new locations list

In [4]:
loc=paste(loc$X0, ":", loc$X1, "-", loc$X2, sep="")

In [5]:
head(loc)

[1] "211000022278049:390-394"   "211000022278049:537-541"  
[3] "211000022278049:573-577"   "211000022278049:837-841"  
[5] "211000022278049:1613-1617" "211000022278049:1618-1622"

In [6]:
head(metadata)

,X,Rep,Tissue,Country,Line,NotSureProtein,protein
1,damID-Dam_Fr188_C1,1,C,Fr,188,damID,Dam
2,damID-Dam_Fr188_C2,2,C,Fr,188,damID,Dam
3,damID-Dam_Fr188_C3,3,C,Fr,188,damID,Dam
4,damID-Dam_Fr188_C4,4,C,Fr,188,damID,Dam
5,damID-Dam_Fr188_T3,3,T,Fr,188,damID,Dam
6,damID-Dam_Fr188_T4,4,T,Fr,188,damID,Dam


In [7]:
unique(metadata$Rep)
unique(metadata$Tissue)
unique(metadata$Country)
unique(metadata$Line)
unique(metadata$NotSureProtein) #this is batch
unique(metadata$protein)

[1]  1  2  3  4 NA

[1] C T W w
Levels: C T w W

[1] Fr Zi
Levels: Fr Zi

[1] 188 257 275  89 238

[1] damID  damID2 damID3
Levels: damID damID2 damID3

[1] Dam Lam D1 
Levels: D1 Dam Lam

What is line 275?  It is: 'damID-Dam_Zi257_T1'.  Confirmed that it's just a typo, and means 257.  There is already a replicate 1 for Dam Zi 257, so we change the value to 'damID-Dam_Zi257_T3'

In [8]:
metadata$X <- as.character(metadata$X)
metadata[metadata$Line == '275',]
metadata[metadata$Line == '257' & metadata$Tissue == 'T' & metadata$protein == 'Dam',]
metadata[11,1]<-'damID-Dam_Zi257_T3'
metadata[11,]$Line<-257
metadata[11,]$Rep<-3
metadata[11,]

,X,Rep,Tissue,Country,Line,NotSureProtein,protein
11,damID-Dam_Zi275_T1,1,T,Zi,275,damID,Dam


,X,Rep,Tissue,Country,Line,NotSureProtein,protein
79,damID3-Dam_Zi257_T1,1,T,Zi,257,damID3,Dam
80,damID3-Dam_Zi257_T2,2,T,Zi,257,damID3,Dam


,X,Rep,Tissue,Country,Line,NotSureProtein,protein
11,damID-Dam_Zi257_T3,3,T,Zi,257,damID,Dam


In [9]:
#Replace the Na reps with 0
metadata[is.na(metadata)] <- 0
unique(metadata$Rep)

#Replace 'w' with 'W'
metadata$Tissue <- toupper(metadata$Tissue)
unique(metadata$Tissue)

#Replace metadata row names to conform to column names in damid (introduced in next cell)
rownames(metadata)=gsub('-', '.', metadata$X)
metadata$X = NULL

#Change to factors
metadata$Country<-factor(metadata$Country)
metadata$Line<-factor(metadata$Line)
metadata$Protein<-factor(metadata$protein)
metadata$Batch<-factor(metadata$NotSureProtein)

#Clean up names
metadata$protein<-NULL
metadata$NotSureProtein<-NULL

head(metadata)

[1] 1 2 3 4 0

[1] "C" "T" "W"

,Rep,Tissue,Country,Line,Protein,Batch
damID.Dam_Fr188_C1,1,C,Fr,188,Dam,damID
damID.Dam_Fr188_C2,2,C,Fr,188,Dam,damID
damID.Dam_Fr188_C3,3,C,Fr,188,Dam,damID
damID.Dam_Fr188_C4,4,C,Fr,188,Dam,damID
damID.Dam_Fr188_T3,3,T,Fr,188,Dam,damID
damID.Dam_Fr188_T4,4,T,Fr,188,Dam,damID


Remove row names that are not batch3 for 'W'

In [10]:
metadata[(metadata$Batch != 'damID3'& metadata$Tissue == 'W'),]
removeWNotBatch3<-c('damID.Dam_Zi257_W','damID2.Dam_Fr188_w','damID2.Dam_Zi257_w')
metadata<-metadata[-which(rownames(metadata) %in% removeWNotBatch3), ]


,Rep,Tissue,Country,Line,Protein,Batch
damID.Dam_Zi257_W,0,W,Zi,257,Dam,damID
damID2.Dam_Fr188_w,0,W,Fr,188,Dam,damID2
damID2.Dam_Zi257_w,0,W,Zi,257,Dam,damID2


damid dataframe has no mitochondrial genome or rDNA, and we only use columns with sum of counts > 500000.

In [11]:
colnames(damid)
#there is a mix of uppercase and lower case W in the colnames, we make them all uppercase
colnames(damid)=gsub('w', 'W', colnames(damid))

#Remove the column with 275: 
#damid$damID.Dam_Zi275_T1<-NULL
colnames(damid)[colnames(damid) == 'damID.Dam_Zi275_T1'] <- 'damID.Dam_Zi257_T3'

#Remove the W's that aren't batch3
damid$damID.Dam_Zi257_W<-NULL
damid$damID2.Dam_Fr188_W<-NULL
damid$damID2.Dam_Zi257_W<-NULL

head(damid)
"dimensions of damid"
dim(damid)

[1] "X"                   "damID.Dam_Fr188_C1"  "damID.Dam_Fr188_C2" 
  [4] "damID.Dam_Fr188_C3"  "damID.Dam_Fr188_C4"  "damID.Dam_Fr188_T3" 
  [7] "damID.Dam_Fr188_T4"  "damID.Dam_Zi257_C1"  "damID.Dam_Zi257_C2" 
 [10] "damID.Dam_Zi257_C3"  "damID.Dam_Zi257_W"   "damID.Dam_Zi275_T1" 
 [13] "damID.Lam_Fr188_C1"  "damID.Lam_Fr188_C2"  "damID.Lam_Fr188_C3" 
 [16] "damID.Lam_Fr188_C"   "damID.Lam_Fr188_T2"  "damID.Lam_Fr188_T3" 
 [19] "damID.Lam_Zi257_C1"  "damID.Lam_Zi257_C2"  "damID.Lam_Zi257_C3" 
 [22] "damID.Lam_Zi257_T1"  "damID.Lam_Zi257_T2"  "damID.Lam_Zi257_T3" 
 [25] "damID2.D1_Fr188_C3"  "damID2.D1_Fr188_T3"  "damID2.D1_Fr89_C2"  
 [28] "damID2.D1_Fr89_C3"   "damID2.D1_Fr89_T3"   "damID2.D1_Zi238_C1" 
 [31] "damID2.D1_Zi238_C3"  "damID2.D1_Zi238_T3"  "damID2.D1_Zi257_C1" 
 [34] "damID2.D1_Zi257_C2"  "damID2.D1_Zi257_C3"  "damID2.D1_Zi257_T2" 
 [37] "damID2.D1_Zi257_T3"  "damID2.Dam_Fr188_w"  "damID2.Dam_Fr89_C1" 
 [40] "damID2.Dam_Fr89_C3"  "damID2.Dam_Fr89_T1"  "damID2.Dam_Fr89_T2" 
 [43] "damID2.Dam_Zi238_C1" "damID2.Dam_Zi238_C2" "damID2.Dam_Zi238_C3"
 [46] "damID2.Dam_Zi257_w"  "damID2.Lam_Fr89_C1"  "damID2.Lam_Fr89_C2" 
 [49] "damID2.Lam_Zi238_C1" "damID2.Lam_Zi238_C2" "damID2.Lam_Zi238_C3"
 [52] "damID2.Lam_Zi257_C4" "damID3.D1_Fr188_T"   "damID3.D1_Fr188_w1" 
 [55] "damID3.D1_Fr188_w2"  "damID3.D1_Fr188_w3"  "damID3.D1_Fr89_w1"  
 [58] "damID3.D1_Fr89_w2"   "damID3.D1_Fr89_w3"   "damID3.D1_Zi238_T"  
 [61] "damID3.D1_Zi238_w1"  "damID3.D1_Zi238_w2"  "damID3.D1_Zi238_w3" 
 [64] "damID3.D1_Zi257_T"   "damID3.D1_Zi257_w1"  "damID3.D1_Zi257_w2" 
 [67] "damID3.D1_Zi257_w3"  "damID3.Dam_Fr188_T2" "damID3.Dam_Fr188_w1"
 [70] "damID3.Dam_Fr188_w2" "damID3.Dam_Fr188_w3" "damID3.Dam_Fr89_w1" 
 [73] "damID3.Dam_Fr89_w2"  "damID3.Dam_Fr89_w3"  "damID3.Dam_Zi238_T1"
 [76] "damID3.Dam_Zi238_T2" "damID3.Dam_Zi238_w1" "damID3.Dam_Zi238_w2"
 [79] "damID3.Dam_Zi238_w3" "damID3.Dam_Zi257_T1" "damID3.Dam_Zi257_T2"
 [82] "damID3.Dam_Zi257_w1" "damID3.Dam_Zi257_w2" "damID3.Dam_Zi257_w3"
 [85] "damID3.Lam_Fr188_T1" "damID3.Lam_Fr188_T4" "damID3.Lam_Fr188_w1"
 [88] "damID3.Lam_Fr188_w2" "damID3.Lam_Fr188_w3" "damID3.Lam_Fr89_T1" 
 [91] "damID3.Lam_Fr89_T2"  "damID3.Lam_Fr89_w1"  "damID3.Lam_Fr89_w2" 
 [94] "damID3.Lam_Fr89_w3"  "damID3.Lam_Zi238_w1" "damID3.Lam_Zi238_w2"
 [97] "damID3.Lam_Zi238_w3" "damID3.Lam_Zi257_w1" "damID3.Lam_Zi257_w2"
[100] "damID3.Lam_Zi257_w3"

,X,damID.Dam_Fr188_C1,damID.Dam_Fr188_C2,damID.Dam_Fr188_C3,damID.Dam_Fr188_C4,damID.Dam_Fr188_T3,damID.Dam_Fr188_T4,damID.Dam_Zi257_C1,damID.Dam_Zi257_C2,damID.Dam_Zi257_C3,ellip.h,damID3.Lam_Fr89_T2,damID3.Lam_Fr89_W1,damID3.Lam_Fr89_W2,damID3.Lam_Fr89_W3,damID3.Lam_Zi238_W1,damID3.Lam_Zi238_W2,damID3.Lam_Zi238_W3,damID3.Lam_Zi257_W1,damID3.Lam_Zi257_W2,damID3.Lam_Zi257_W3
1,0,3,1,0,0,3,3,1,2,0,⋯,1,0,1,2,1,0,0,0,1,0
2,1,3,0,0,0,2,3,1,1,0,⋯,1,0,0,1,0,1,0,1,0,0
3,2,3,0,0,0,2,3,1,1,0,⋯,1,0,0,1,0,1,1,1,0,0
4,3,1,0,0,0,2,1,2,1,1,⋯,1,0,2,0,0,1,1,1,0,0
5,4,0,2,2,2,0,1,1,2,0,⋯,1,1,1,1,0,0,1,0,1,2
6,5,0,2,2,2,0,1,1,2,0,⋯,1,1,1,1,0,0,1,0,1,2


[1] "dimensions of damid"

[1] 388464     97

### Extracting information for each line by protein by line combination in DESeq

We want to separate tissues, because that's what we did in our analysis of gene expression data.

The reason we run DESeq independently for each line is because we want the shrinkages used in the log2FoldChange calculation to be independent.  (Because of shrinkage, the log2foldchange might be more trustworthy than baseMean (ie. mean(D1)/mean(DAM) (or is it the inverse?)).

The following cell returns a list of files of the form:
paste("results_", tissue, "_", protein,"_", line, sep="")
which has all the information "baseMean","log2FoldChange","lfcSE","stat","pvalue","padj".

There are some combinations, like T 238 that can't be used for this analysis (no Lam or D1 entries for T 238, just Dam).


In [14]:
return_dataset_for_plot<-function(tissue,protein,line, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))
    colsLine<-c(grep(line, names(damid)))
    
    cols<-intersect(intersect(colsTissue,colsLine),union(colsDam,colsProtein))
    
    print(cols)

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam") & metadata$Line==line,]

    print(metadata)
    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein)
    suppressWarnings(dds<-DESeq(dataset))
    res.dds<-results(dds)
    write.csv(res.dds, file=paste("results_", tissue, "_", protein,"_", line, sep=""))
}


#There are some combinations, like T 238 that can't be used for this analysis (no Lam or D1 entries)
for (tissue in c("C", "W","T")) {
    for (line in c("238","257","89","188")) {
        for (protein in c("Lam", "D1")) {
            try(return_dataset_for_plot(tissue, protein, line, damid, metadata))
        }
    }
}


[1] 41 42 43 46 47 48
                    Rep Tissue Country Line Protein  Batch
damID2.Dam_Zi238_C1   1      C      Zi  238     Dam damID2
damID2.Dam_Zi238_C2   2      C      Zi  238     Dam damID2
damID2.Dam_Zi238_C3   3      C      Zi  238     Dam damID2
damID2.Lam_Zi238_C1   1      C      Zi  238     Lam damID2
damID2.Lam_Zi238_C2   2      C      Zi  238     Lam damID2
damID2.Lam_Zi238_C3   3      C      Zi  238     Lam damID2


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] 29 30 41 42 43
                    Rep Tissue Country Line Protein  Batch
damID2.D1_Zi238_C1    1      C      Zi  238      D1 damID2
damID2.D1_Zi238_C3    3      C      Zi  238      D1 damID2
damID2.Dam_Zi238_C1   1      C      Zi  238     Dam damID2
damID2.Dam_Zi238_C2   2      C      Zi  238     Dam damID2
damID2.Dam_Zi238_C3   3      C      Zi  238     Dam damID2


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1]  8  9 10 18 19 20 49
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Zi257_C1    1      C      Zi  257     Dam  damID
damID.Dam_Zi257_C2    2      C      Zi  257     Dam  damID
damID.Dam_Zi257_C3    3      C      Zi  257     Dam  damID
damID.Lam_Zi257_C1    1      C      Zi  257     Lam  damID
damID.Lam_Zi257_C2    2      C      Zi  257     Lam  damID
damID.Lam_Zi257_C3    3      C      Zi  257     Lam  damID
damID2.Lam_Zi257_C4   4      C      Zi  257     Lam damID2


...

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1]  6  7 16 17 65 82 83
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Fr188_T3    3      T      Fr  188     Dam  damID
damID.Dam_Fr188_T4    4      T      Fr  188     Dam  damID
damID.Lam_Fr188_T2    2      T      Fr  188     Lam  damID
damID.Lam_Fr188_T3    3      T      Fr  188     Lam  damID
damID3.Dam_Fr188_T2   2      T      Fr  188     Dam damID3
damID3.Lam_Fr188_T1   1      T      Fr  188     Lam damID3
damID3.Lam_Fr188_T4   4      T      Fr  188     Lam damID3


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1]  6  7 25 50 65
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Fr188_T3    3      T      Fr  188     Dam  damID
damID.Dam_Fr188_T4    4      T      Fr  188     Dam  damID
damID2.D1_Fr188_T3    3      T      Fr  188      D1 damID2
damID3.D1_Fr188_T     0      T      Fr  188      D1 damID3
damID3.Dam_Fr188_T2   2      T      Fr  188     Dam damID3


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


## Extracting main effects info (also info about batch and interactions)

Here we don't separate information by line.  These give us general results on D1 and Lam, and tell us about the effect of Batch.  

The conclusion we get from doing a binomial test for carcass and D1: (full = ~Line*Protein; reduced= ~Line + Protein)  and (full = ~Line + Protein; reduced= ~Protein):
D1 protein binds to few places. We don't see a main effect for D1, so we also don’t predict to be able to see an interaction of line:protein (and we don't). However, the fact that there isn't a high percentage of LFC >0 and LFC <0  doesn’t mean there isn’t differential binding of D1. (Low counts= 83%).  If the (few) sites at which we see differential binding do  have differential gene expression by line, then this would point to a possible mechanism by which the Y chromosome can affect gene expression across the autosomes.  This is because we know that D1 binds to several Y chromosome satellites.  So this would be consistent with the hypothesis that the Y chromosome acts as a protein sink.
We can't test Batch for D1 because not enough variety of Batch for D1 protein.


The conclusion we get from doing a binomial test for carcass and Lam: (full = ~Line*Protein; reduced= ~Line + Protein) and (full = ~Line + Protein; reduced= ~Protein) and (full = ~Line*Protein + Batch; reduced= ~Line*Protein):
1. We see more binding of the Lam protein than D1 protein (this is expected).  Specifically, more main effects of D1, and we also see evidence for the interaction Line:protein.  We might be able to conclude from this that there is a link between variation in expression and variation in chromatin state (once we do that analysis), but unlike in the D1 case we can't draw conclusions about the mechanism.  We can simply conclude that some underlying genetic mechanism on Y chromosome affects both changes in expression and changes in chromatin.  It could be for example Y chromo affects expression which in turn affects chromatin state, or Y chromo affects chromatin state which in turn affects gene expression.  It could also be structural (different sites are near the nuclear envelope depending on the size of the Y chromosome).
2.  Batch doesn't seem to have much of an effect, at least in the carcass and Lam case.

Whar do we make of the fact that we dont see diff expression for W?



We first run a control: we restrict ourselves to Dam-only data, and check to see if an LRT prefers full model: ~Line over reduced model: ~1.  Then answer is that for carcass data, there does seem to be a line effect.  Not sure what to make of it.

In [15]:
interaction_Dam_control<-function(tissue, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,colsDam)
    print(cols)

    protein="Dam"
    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Line)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~1)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    
    #write.csv(res.ddsLRT, file=paste("nbin_interaction_results_", tissue, "_", protein, sep=""))
    
    #res.dds<-results(dds)
    #write.csv(res.dds, file=paste("interaction_results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    print(tissue)
    interaction_Dam_control(tissue, damid, metadata)
}



[1] "C"
 [1]  2  3  4  5  8  9 10 37 38 41 42 43


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388415 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 3906, 1% 
LFC < 0 (down)   : 3888, 1% 
outliers [1]     : 17, 0.0044% 
low counts [2]   : 278626, 72% 
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"
 [1] 66 67 68 69 70 71 74 75 76 79 80 81


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388376 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 8, 0.0021% 
LFC < 0 (down)   : 56, 0.014% 
outliers [1]     : 0, 0% 
low counts [2]   : 301186, 78% 
(mean count < 7)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"
 [1]  6  7 11 39 40 65 72 73 77 78


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 387327 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 3474, 0.9% 
LFC < 0 (down)   : 3580, 0.92% 
outliers [1]     : 59, 0.015% 
low counts [2]   : 322853, 83% 
(mean count < 5)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



These values for the control are quite high in testes and carcass, especially in comparison to whole-fly tissue.  We do the same analysis with Lam and D1, ignoring Dam.

In [12]:
#Here we do control with Lam
interaction_Dam_control<-function(tissue, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsLam<-c(grep("Lam_", names(damid)))

    cols<-intersect(colsTissue,colsLam)
    print(cols)

    protein="Lam"
    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Lam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Line)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~1)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    
    #write.csv(res.ddsLRT, file=paste("nbin_interaction_results_", tissue, "_", protein, sep=""))
    
    #res.dds<-results(dds)
    #write.csv(res.dds, file=paste("interaction_results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    print(tissue)
    interaction_Dam_control(tissue, damid, metadata)
}



[1] "C"
 [1] 12 13 14 15 18 19 20 44 45 46 47 48 49


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388446 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 23333, 6% 
LFC < 0 (down)   : 29712, 7.6% 
outliers [1]     : 2701, 0.7% 
low counts [2]   : 97904, 25% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"
 [1] 84 85 86 89 90 91 92 93 94 95 96 97


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388427 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 255, 0.066% 
LFC < 0 (down)   : 1148, 0.3% 
outliers [1]     : 0, 0% 
low counts [2]   : 369003, 95% 
(mean count < 22)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"
[1] 16 17 21 22 23 82 83 87 88


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388382 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 65, 0.017% 
LFC < 0 (down)   : 11, 0.0028% 
outliers [1]     : 988, 0.25% 
low counts [2]   : 218348, 56% 
(mean count < 6)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [13]:
#Here we do control with D1
interaction_Dam_control<-function(tissue, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsD1<-c(grep("D1_", names(damid)))

    cols<-intersect(colsTissue,colsD1)
    print(cols)

    protein="D1"
    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="D1"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Line)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~1)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    
    #write.csv(res.ddsLRT, file=paste("nbin_interaction_results_", tissue, "_", protein, sep=""))
    
    #res.dds<-results(dds)
    #write.csv(res.dds, file=paste("interaction_results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    print(tissue)
    interaction_Dam_control(tissue, damid, metadata)
}




[1] "C"
[1] 24 26 27 29 30 32 33 34


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 384975 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 235, 0.061% 
LFC < 0 (down)   : 1512, 0.39% 
outliers [1]     : 0, 0% 
low counts [2]   : 350631, 91% 
(mean count < 6)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"
 [1] 51 52 53 54 55 56 58 59 60 62 63 64


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388341 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 3, 0.00077% 
LFC < 0 (down)   : 6, 0.0015% 
outliers [1]     : 1, 0.00026% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"
[1] 25 28 31 35 36 50 57 61


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 383232 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1921, 0.5% 
LFC < 0 (down)   : 2457, 0.64% 
outliers [1]     : 766, 0.2% 
low counts [2]   : 363808, 95% 
(mean count < 11)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



Here we test interactions, ie full model :~Protein + Line + Protein:Line, and reduced model : ~Protein + Line.

In [12]:
return_dataset_for_plot2_interaction<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line + Line:Protein)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~Line + Protein)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    
    write.csv(res.ddsLRT, file=paste("nbin_interaction_results_", tissue, "_", protein, sep=""))
    
    #res.dds<-results(dds)
    #write.csv(res.dds, file=paste("interaction_results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    for (protein in c( "D1", "Lam")) {
        print(c(tissue, protein))
        try(return_dataset_for_plot2_interaction(tissue, protein, damid, metadata))
        #return_dataset_for_plot2_interaction(tissue, protein, damid, metadata)
    }
}


[1] "C"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388463 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2485, 0.64% 
LFC < 0 (down)   : 1628, 0.42% 
outliers [1]     : 14, 0.0036% 
low counts [2]   : 323851, 83% 
(mean count < 5)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "C"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 24281, 6.3% 
LFC < 0 (down)   : 28593, 7.4% 
outliers [1]     : 887, 0.23% 
low counts [2]   : 120500, 31% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"  "D1"


...

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388463 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 120, 0.031% 
LFC < 0 (down)   : 148, 0.038% 
outliers [1]     : 0, 0% 
low counts [2]   : 346444, 89% 
(mean count < 13)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
1 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest
found results columns, replacing these
1 rows did not converge in beta, labelled in mcols(object)$fullBetaConv. Use larger maxit argument with nbinomLRT


[1] "summary LRT"

out of 388404 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2330, 0.6% 
LFC < 0 (down)   : 2021, 0.52% 
outliers [1]     : 2816, 0.73% 
low counts [2]   : 361295, 93% 
(mean count < 10)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"   "Lam"


factor levels were dropped which had no samples


In [13]:
#We couldn't run T Lam, because there are no 238/Lam columns, so the matrix model is not full rank. 
#Here we run T Lam

return_dataset_for_plot2_interaction_TLam<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))
    
    #all lines but not 238
    colsLine<-c(grep("257", names(damid)), grep("188", names(damid)),grep("89", names(damid)))

    cols<-intersect(intersect(colsTissue, colsLine),union(colsDam,colsProtein))
    
    #also remove 238
    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam") & metadata$Line!='238',]
    
    print(dim(damid[,cols]))
    print(dim(metadata))
    print(metadata)
    
    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line + Line:Protein)
    
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~Line + Protein)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    
    write.csv(res.ddsLRT, file=paste("nbin_interaction_results_", tissue, "_", protein, sep=""))
    
    #res.dds<-results(dds)
    #write.csv(res.dds, file=paste("interaction_results_", tissue, "_", protein, sep=""))
}


return_dataset_for_plot2_interaction_TLam('T', 'Lam', damid, metadata)






[1] 388464     17
[1] 17  6
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Fr188_T3    3      T      Fr  188     Dam  damID
damID.Dam_Fr188_T4    4      T      Fr  188     Dam  damID
damID.Dam_Zi257_T3    3      T      Zi  257     Dam  damID
damID.Lam_Fr188_T2    2      T      Fr  188     Lam  damID
damID.Lam_Fr188_T3    3      T      Fr  188     Lam  damID
damID.Lam_Zi257_T1    1      T      Zi  257     Lam  damID
damID.Lam_Zi257_T2    2      T      Zi  257     Lam  damID
damID.Lam_Zi257_T3    3      T      Zi  257     Lam  damID
damID2.Dam_Fr89_T1    1      T      Fr   89     Dam damID2
damID2.Dam_Fr89_T2    2      T      Fr   89     Dam damID2
damID3.Dam_Fr188_T2   2      T      Fr  188     Dam damID3
damID3.Dam_Zi257_T1   1      T      Zi  257     Dam damID3
damID3.Dam_Zi257_T2   2      T      Zi  257     Dam damID3
damID3.Lam_Fr188_T1   1      T      Fr  188     Lam damID3
damID3.Lam_Fr188_T4   4      T      Fr  188     Lam damID3
damID3.Lam_Fr89_T1    1     

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388456 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1211, 0.31% 
LFC < 0 (down)   : 1763, 0.45% 
outliers [1]     : 368, 0.095% 
low counts [2]   : 316110, 81% 
(mean count < 8)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



1) It was confusing to see Dam control behave so different in C/T vs W, but if we look at Lam only and D1 only, they behave proportionally to when we test the protein:line interaction, which is good because it means that we aren't attributing differential binding to only variatino in Dam.  
2) Also, generally we see a bit fewer significant interactions then in the protein-only case, which is what we would expect to observe, because some of the variability is being normalized out by the Dam control.

Weirdness in Dam control could be due to general variation in accessibility in different lines.  But it's at least not creating false positives, it's not inflating the number of differential binding regions that we observe, which would be the biggest concern.

Here we test main effects

In [43]:
return_dataset_for_plot2_mainEffect<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line)
    suppressWarnings(dds<-DESeq(dataset))

    ddsLRT<-nbinomLRT(dds, reduced= ~Line)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    #write.csv(res.dds, file=paste("results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    for (protein in c("Lam", "D1")) {
        print(c(tissue, protein))
        #try(return_dataset_for_plot2_interaction(tissue, protein, line, damid, metadata))
        return_dataset_for_plot2_mainEffect(tissue, protein, damid, metadata)
    }
}



[1] "C"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 72152, 19% 
LFC < 0 (down)   : 65362, 17% 
outliers [1]     : 3574, 0.92% 
low counts [2]   : 37656, 9.7% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "C"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388463 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2789, 0.72% 
LFC < 0 (down)   : 4433, 1.1% 
outliers [1]     : 68, 0.018% 
low counts [2]   : 248531, 64% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"   "Lam"


...

[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 19550, 5% 
LFC < 0 (down)   : 20629, 5.3% 
outliers [1]     : 673, 0.17% 
low counts [2]   : 105440, 27% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388456 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 56168, 14% 
LFC < 0 (down)   : 62316, 16% 
outliers [1]     : 1282, 0.33% 
low counts [2]   : 67780, 17% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388404 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 141, 0.036% 
LFC < 0 (down)   : 195, 0.05% 
outliers [1]     : 6433, 1.7% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



Testing batch

In [77]:
#C/D1 might not have enough diversity in batch to test batch effect

return_dataset_for_plot2_batch<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line + Line:Protein + Batch)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~Line + Protein + Line:Protein)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    #write.csv(res.dds, file=paste("results_", tissue, "_", protein, sep=""))
}

#We do not test 'W' because there is only one batch, batch3 (we removed 'W' rows that were batch1 and batch2 because batch was significant for these)
for (tissue in c("C", "T")) {
    for (protein in c("Lam", "D1")) {
        print(c(tissue, protein))
        #try(return_dataset_for_plot2_interaction(tissue, protein, line, damid, metadata))
        try(return_dataset_for_plot2_batch(tissue, protein, damid, metadata))
    }
}



[1] "C"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1688, 0.43% 
LFC < 0 (down)   : 6433, 1.7% 
outliers [1]     : 703, 0.18% 
low counts [2]   : 218409, 56% 
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "C"  "D1"


factor levels were dropped which had no samples


[1] "W"   "Lam"


...

[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 41357, 11% 
LFC < 0 (down)   : 35821, 9.2% 
outliers [1]     : 0, 0% 
low counts [2]   : 135566, 35% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"   "Lam"


factor levels were dropped which had no samples


[1] "T"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
633 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest
found results columns, replacing these
633 rows did not converge in beta, labelled in mcols(object)$fullBetaConv. Use larger maxit argument with nbinomLRT


[1] "summary LRT"

out of 388359 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1901, 0.49% 
LFC < 0 (down)   : 5077, 1.3% 
outliers [1]     : 0, 0% 
low counts [2]   : 323760, 83% 
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

